In [124]:
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join
import re
import numpy as np
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb
import string


def print_mat(mat):
    for row in mat:
        print(''.join(row))


files = [f for f in listdir('.') if isfile(join('.', f))]
pat = re.compile('(ex(\d)+.txt)|(in(\d)*.txt)')
files = [f for f in files if pat.match(f)]

box = widgets.Select(
    options=files,
    value='ex1.txt',
    rows=4,
    description='Input file:',
    disabled=False
)

display(box)


Select(description='Input file:', index=1, options=('in.txt', 'ex1.txt'), rows=4, value='ex1.txt')

## Parse file

In [125]:
lines = list(map(lambda l: l.rstrip(), open(box.value, 'r').readlines()))

P = [[tuple(map(int, p.split(','))) for p in line.split(' -> ')]
     for line in lines]

minx = np.inf
miny = 0
maxx = -np.inf
maxy = -np.inf
for p in P:
    for x, y in p:
        minx = min(minx, x)
        maxx = max(maxx, x)
        maxy = max(maxy, y)

P = list([list(map(lambda q: (q[0] - minx, q[1]), p)) for p in P])

GRID = np.full((maxx - minx + 1, maxy + 1), '.')

START = (500 - minx, 0)
GRID[START] = '+'

for p in P:
    for i in range(len(p)-1):
        x, y = p[i]
        u, v = p[i+1]
        if x == u:
            GRID[x, min(y, v):max(y, v)+1] = '#'
        else:
            GRID[min(x, u):max(x, u)+1, y] = '#'


print("Done parsing \"{name}\"".format(name=box.value))


Done parsing "in.txt"


### Part 1

In [126]:
def sand(G):
    p = START
    while True:
        x, y = p
        if y >= maxy:
            return False
        if G[x, y+1] == '.':
            p = (x, y+1)
        elif G[x-1, y+1] == '.':
            p = (x-1, y+1)
        elif G[x+1, y+1] == '.':
            p = (x+1, y+1)
        else:
            break

    G[p] = 'o'
    return True


G = deepcopy(GRID)
while sand(G):
    pass

print(np.sum(G == 'o'))


610


### Part 2

In [127]:
def sand(G):
    p = (START[0] + 2*maxy, START[1])
    while True:
        x, y = p
        if G[x, y+1] == '.':
            p = (x, y+1)
        elif x >= 0 and G[x-1, y+1] == '.':
            p = (x-1, y+1)
        elif x < G.shape[0] and G[x+1, y+1] == '.':
            p = (x+1, y+1)
        else:
            if y == 0:
                return False
            break

    G[p] = 'o'
    return True


G2 = deepcopy(GRID)
G2 = np.vstack((G2, np.full((maxy*2, G2.shape[1]), '.')))
G2 = np.vstack((np.full((maxy*2, G2.shape[1]), '.'), G2))
G2 = np.hstack((G2, np.full((G2.shape[0], 2), '.')))
G2[:, maxy+2] = '#'


while sand(G2):
    pass

print(np.sum(G2 == 'o') + 1)


27194
